In [1]:
# general imports
from e2e_model import E2EModel

import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

# --- CORRECTED Sionna imports ---

# PlotBER is in sionna.utils
from sionna.phy.utils.plotting import PlotBER
from sionna.phy.fec.linear import LinearEncoder


# LDPC Encoders and Decoders
from sionna.phy.fec.ldpc import LDPCBPDecoder, LDPC5GEncoder, LDPC5GDecoder
# General FEC utility
from sionna.phy.fec.utils import gm2pcm, load_parity_check_examples

# --- End of corrected imports ---


%load_ext autoreload
%autoreload 2
from gnn import * # load GNN functions
from wbp import * # load weighted BP functions

print(f"All modules loaded successfully!")

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
gpus = tf.config.list_physical_devices('GPU')
print('Number of GPUs available :', len(gpus))
if gpus:
    gpu_num = 1 # Number of the GPU to be used
    try:
        #tf.config.set_visible_devices([], 'GPU')
        tf.config.set_visible_devices(gpus[gpu_num], 'GPU')
        print('Only GPU number', gpu_num, 'used.')
        tf.config.experimental.set_memory_growth(gpus[gpu_num], True)
    except RuntimeError as e:
        print(e)

Number of GPUs available : 0


In [4]:
tf.config.list_physical_devices


<function tensorflow.python.framework.config.list_physical_devices(device_type=None)>

In [4]:
#----- LDPC 5G -----
params={
    # --- Code Parameters ---
        "code": "5G-LDPC",
        "n": 140,
        "k": 60,
    # --- GNN Architecture ----
        "num_embed_dims": 16,
        "num_msg_dims": 16,
        "num_hidden_units": 48,
        "num_mlp_layers": 3,
        "num_iter": 10,
        "reduce_op": "sum",
        "activation": "relu",
        "clip_llr_to": 20,
        "use_attributes": False,
        "node_attribute_dims": 0,
        "msg_attribute_dims": 0,
        "return_infobits": False,
        "use_bias": True,
    # --- Training ---- #
        "batch_size": [128, 128, 128], # bs, iter, lr must have same dim
        "train_iter": [35000, 300000, 300000],
        "learning_rate": [5e-4, 1e-4, 1e-5],
        "ebno_db_train": [2, 8.],
        "ebno_db_eval": 2.,
        "batch_size_eval": 1000, # batch size only used for evaluation during training
        "eval_train_steps": 1000, # evaluate model every N iters
    # --- Log ----
        "save_weights_iter": 10000, # save weights every X iters
        "run_name": "LDPC_5G_01", # name of the stored weights/logs
        "save_dir": "results/", # folder to store results
    # --- MC Simulation parameters ----
        "eval_num_iter": 10, # number of decoding iters to evaluate
        "mc_iters": 100,
        "mc_batch_size": 1000,
        "num_target_block_errors": 500,
        "ebno_db_min": 0.,
        "ebno_db_max": 4.5,
        "ebno_db_stepsize": 0.5,
        "eval_ns": [140, 280, 420, 280, 280, 280], # evaluate different lengths
        "eval_ks": [60, 120, 180, 120, 90, 150],
        "sim_esno": False, # simulate results in EsN0
}


In [5]:
# all codes must provide an encoder-layer and a pcm
if params["code"]=="5G-LDPC":
    print("Loading 5G NR LDPC code")

    k = params["k"]
    n = params["n"]

    encoder_5g = LDPC5GEncoder(k, n)
    decoder_5g = LDPC5GDecoder(encoder_5g,
                               num_iter=params["eval_num_iter"],
                               return_infobits=False,
                               prune_pcm=True)

    pcm,_ = generate_pruned_pcm_5g(decoder_5g, n)

    n_no_rm = pcm.shape[1]
    k_no_rm = pcm.shape[1] - pcm.shape[0]

    # create encoder without rate-matching
    u_ref = np.eye(k)
    c_ref = encoder_5g(u_ref).numpy()
    gm = np.concatenate([u_ref[:,:2*encoder_5g._z], c_ref], axis=1)
    encoder_no_rm = LinearEncoder(gm, is_pcm=False)

else:
    raise ValueError("Unknown code type")

Loading 5G NR LDPC code


AlreadyExistsError: {{function_node __wrapped__Pack_N_1_device_/job:localhost/replica:0/task:0/device:GPU:0}} TensorFlow device (GPU:0) is being mapped to multiple devices (0 now, and 1 previously), which is not supported. This may be the result of providing different GPU configurations (ConfigProto.gpu_options, for example different visible_device_list) when creating multiple Sessions in the same process. This is not currently supported, see https://github.com/tensorflow/tensorflow/issues/19083 [Op:Pack] name: stack

In [ ]:
ber_plot = PlotBER(f"GNN-based Decoding - {params['code']}, (k,n)=({k},{n})")
ebno_dbs = np.arange(params["ebno_db_min"],
                     params["ebno_db_max"]+1,
                     params["ebno_db_stepsize"])

In [ ]:
# --- Uncoded QPSK BER Simulation (Sionna 1.2.1) ---

import tensorflow as tf
from e2e_model import E2EModel

# Use earlier k,n if defined; otherwise default
k = k if 'k' in globals() else 100
n = n if 'n' in globals() else 100

# Create uncoded model
e2e_uncoded = E2EModel(None, None, k=k, n=n, modulation="qam", num_bits_per_symbol=2)

# --- Soft-output (LLR) Monte Carlo function ---
def mc_fun_soft(batch_size, ebno_db):
    """For soft_estimates=True: returns (bits, LLRs)"""
    b = tf.random.uniform([batch_size, k], maxval=2, dtype=tf.int32)
    llr = e2e_uncoded(b, ebno_db=ebno_db, return_llr=True)
    return b, llr

# --- Hard-decision Monte Carlo function ---
def mc_fun_hard(batch_size, ebno_db):
    """For soft_estimates=False: returns (bits, hard decisions)"""
    b = tf.random.uniform([batch_size, k], maxval=2, dtype=tf.int32)
    b_hat = e2e_uncoded(b, ebno_db=ebno_db, return_llr=False)
    return b, b_hat

# --- Simulate both curves ---
ber_plot.simulate(
    mc_fun_soft,
    ebno_dbs=ebno_dbs,
    batch_size=params["mc_batch_size"],
    num_target_block_errors=params["num_target_block_errors"],
    legend="Uncoded (Soft LLR)",
    soft_estimates=True,
    max_mc_iter=params["mc_iters"],
    forward_keyboard_interrupt=False,
    show_fig=False,
)

ber_plot.simulate(
    mc_fun_hard,
    ebno_dbs=ebno_dbs,
    batch_size=params["mc_batch_size"],
    num_target_block_errors=params["num_target_block_errors"],
    legend="Uncoded (Hard Decision)",
    soft_estimates=False,
    max_mc_iter=params["mc_iters"],
    forward_keyboard_interrupt=False,
    show_fig=False,
)




In [ ]:
# Inspect one entry of ber_plot._bers to see what it contains
for i, entry in enumerate(ber_plot._bers):
    print(f"\nEntry {i}: type = {type(entry)}")
    if isinstance(entry, (list, tuple)):
        print(f"  Length = {len(entry)}")
        for j, item in enumerate(entry):
            print(f"   [{j}] type = {type(item)}, shape = {getattr(item, 'shape', None)}")
    else:
        print(entry)



In [ ]:
# Let's inspect what's inside ber_plot._bers to see the real structure
for i, entry in enumerate(ber_plot._bers):
    print(f"\nEntry {i}: type = {type(entry)}")
    if isinstance(entry, dict):
        print("  Keys:", list(entry.keys()))
        for k, v in entry.items():
            print(f"   {k}: type={type(v)}, shape={getattr(v, 'shape', None)}")
    elif isinstance(entry, (list, tuple)):
        print(f"  Length = {len(entry)}")
        for j, item in enumerate(entry):
            print(f"   [{j}] type = {type(item)}, shape = {getattr(item, 'shape', None)}")
    else:
        print("  Value:", entry)


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

ebno_db = tf.convert_to_tensor(ebno_dbs).numpy()

plt.figure(figsize=(7,5))

# one tensor per legend (same length)
for ber_tensor, legend in zip(ber_plot._bers, ber_plot._legends):
    ber = tf.convert_to_tensor(ber_tensor).numpy()
    plt.semilogy(ebno_db, ber, marker="o", label=legend)
ber_plot._bers.clear()
ber_plot._legends.clear()


plt.xlabel("Eb/N0 [dB]")
plt.ylabel("Bit Error Rate (BER)")
plt.title("Uncoded QPSK BER Curves")
plt.grid(True, which="both")
plt.legend()
plt.show()


In [ ]:
tf.random.set_seed(2) # we fix the seed to ensure stable convergence

# init the GNN decoder
gnn_decoder = GNN_BP(pcm=pcm,
                     num_embed_dims=params["num_embed_dims"],
                     num_msg_dims=params["num_msg_dims"],
                     num_hidden_units=params["num_hidden_units"],
                     num_mlp_layers=params["num_mlp_layers"],
                     num_iter=params["num_iter"],
                     reduce_op=params["reduce_op"],
                     activation=params["activation"],
                     output_all_iter=True,
                     clip_llr_to=params["clip_llr_to"],
                     use_attributes=params["use_attributes"],
                     node_attribute_dims=params["node_attribute_dims"],
                     # --- This line is now fixed ---
                     msg_attribute_dims=params["msg_attribute_dims"],
                     use_bias=params["use_bias"])

# This is the 'model' you will use for training.

In [ ]:
import tensorflow as tf

print("Forcing the GNN layer to build (passing a correctly-shaped empty tensor)...")

# 1. Define the input shape
fake_llr_shape = (1, 160) # (batch_size=1, n_no_rm=160)

# 2. Define the shape for the "empty" attributes tensor
# (batch_size, num_nodes, num_attribute_dims)
fake_attr_shape = (1, 160, 0) # num_attribute_dims is 0

try:
    # 3. Create the two fake tensors
    fake_llr_tensor = tf.ones(fake_llr_shape, dtype=tf.float32)
    
    # This is the key fix: an empty tensor with the right shape and type.
    fake_attr_tensor = tf.zeros(fake_attr_shape, dtype=tf.float32)
    
    # 4. Call the layer with a TUPLE of the two tensors
    _ = gnn_decoder((fake_llr_tensor, fake_attr_tensor))
    
    print("\nCall successful! Model should be built.")

except Exception as e:
    print(f"\nAn error occurred while trying to call the model: {e}")
    print("If this fails, restart the kernel and run all cells from the top.")

# 5. Now, check the build status and print the parameters
print(f"\nModel '{gnn_decoder.name}' is built: {gnn_decoder.built}")
print("Trainable Parameters:")
print("-" * 30)

if not gnn_decoder.trainable_weights:
    print("No trainable weights found. (This is still a problem)")
else:
    total_params = 0
    for param in gnn_decoder.trainable_weights:
        # Added ljust for better formatting
        print(f"Name: {param.name.ljust(60)} | Shape: {param.shape}")
        total_params += tf.size(param).numpy()
    
    print("-" * 30)
    print(f"Total trainable parameters: {total_params}")

In [ ]:
import tensorflow as tf
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import time

# --- Sionna 1.2.1 components ---
from sionna.phy.utils.misc import ebnodb2no
from sionna.phy.mapping import Mapper, Demapper
from sionna.phy.channel import RayleighBlockFading
from sionna.phy.utils.metrics import compute_ber

# --- Constants from your setup ---
k_no_rm = pcm.shape[1] - pcm.shape[0]  # e.g., 100
n_no_rm = pcm.shape[1]                 # e.g., 160
coderate = k_no_rm / n_no_rm
num_iter = params["num_iter"]          # GNN iterations

# --- Components for the simulation ---
# Using BPSK (num_bits_per_symbol = 1)
mapper = Mapper("pam", num_bits_per_symbol=1)
demapper = Demapper("app", "pam", num_bits_per_symbol=1)

# Replace BinarySource with TensorFlow-based generator
def binary_source(batch_size, k):
    """Generate random binary sequences [batch_size, k]."""
    return tf.random.uniform([batch_size, k], maxval=2, dtype=tf.int32)

bce = BinaryCrossentropy(from_logits=True)

# --- Define the Fading Channel ---
# ✅ Correct constructor signature for Sio


In [ ]:
# ===============================
#  IMPORTS & SETUP
# ===============================
import tensorflow as tf
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
import numpy as np
import os, time

# --- Sionna imports ---
from sionna.phy.utils.misc import ebnodb2no
from sionna.phy.utils.metrics import compute_ber
from sionna.phy.mapping import Mapper, Demapper

# ===============================
#  CONSTANTS & PARAMETERS
# ===============================
k_no_rm = pcm.shape[1] - pcm.shape[0]    # message bits
n_no_rm = pcm.shape[1]                   # codeword bits
coderate = k_no_rm / n_no_rm
num_iter = params["num_iter"]            # number of GNN iterations

# ===============================
#  MODEM, LOSS, UTILS
# ===============================
mapper   = Mapper("pam", num_bits_per_symbol=1)
demapper = Demapper("app", "pam", num_bits_per_symbol=1)
bce      = BinaryCrossentropy(from_logits=True)

def binary_source(shape):
    """Generate random bits in {0,1}."""
    return tf.random.uniform(shape, maxval=2, dtype=tf.int32)

def ensure_rank3(t):
    """Ensure tensor has shape [B, N, 1]."""
    return t if t.shape.rank == 3 else t[..., tf.newaxis]

# ===============================
#  ANALYTIC RAYLEIGH + AWGN HELPERS
# ===============================
def sample_rayleigh_block_fading(B, N, complex_channel=True, dtype=tf.float32):
    """Return Rayleigh fading h with shape [B, N, 1]."""
    if complex_channel:
        h_r = tf.random.normal([B, N, 1], mean=0.0, stddev=1/np.sqrt(2), dtype=dtype)
        h_i = tf.random.normal([B, N, 1], mean=0.0, stddev=1/np.sqrt(2), dtype=dtype)
        return tf.complex(h_r, h_i)
    else:
        return tf.random.normal([B, N, 1], mean=0.0, stddev=1.0, dtype=dtype)

def add_awgn(y, no_sym, complex_noise=True):
    """Add AWGN with noise variance N0 per symbol."""
    if complex_noise:
        std = tf.sqrt(no_sym / 2.0)
        n_r = tf.random.normal(tf.shape(y), mean=0.0, stddev=1.0, dtype=y.dtype.real_dtype)
        n_i = tf.random.normal(tf.shape(y), mean=0.0, stddev=1.0, dtype=y.dtype.real_dtype)
        w   = tf.complex(std * n_r, std * n_i)
    else:
        std = tf.sqrt(no_sym)
        w   = std * tf.random.normal(tf.shape(y), mean=0.0, stddev=1.0, dtype=y.dtype)
    return y + w

def llr_perfect_csi(y, h, no_sym):
    """
    Compute per-symbol LLRs with perfect CSI.
    For BPSK: LLR = (2 / N0) * Re{ conj(h) * y }.
    y: [B,N,1], complex or real
    h: [B,N,1], complex or real
    no_sym: [B,1,1] float (N0 per symbol)
    Returns: [B,N,1] float32
    """
    y_c = tf.cast(y, tf.complex64)
    h_c = tf.cast(h, tf.complex64)
    corr = tf.math.real(tf.math.conj(h_c) * y_c)  # float32
    n0 = tf.cast(no_sym, tf.float32)
    llr = (2.0 / n0) * corr
    return tf.cast(llr, tf.float32)

# ===============================
#  OPTIMIZER & CHECKPOINTS
# ===============================
optimizer = Adam(learning_rate=params["learning_rate"][0])
checkpoint_dir = os.path.join(params["save_dir"], params["run_name"], "checkpoints")
os.makedirs(checkpoint_dir, exist_ok=True)

ckpt = tf.train.Checkpoint(model=gnn_decoder)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_dir, max_to_keep=5)

# ===============================
#  TRAIN STEP (fixed)
# ===============================
@tf.function(jit_compile=True)
def train_step(batch_size, ebno_db_range):
    """One training iteration over Rayleigh block fading channel."""
    # 1) Random Eb/N0 in training range
    ebno_db = tf.random.uniform([batch_size],
                                minval=ebno_db_range[0],
                                maxval=ebno_db_range[1],
                                dtype=tf.float32)

    # 2) Generate bits, encode, map
    b = binary_source([batch_size, k_no_rm])  # [B,k]
    c = encoder_no_rm(b)                      # [B,n]
    x = mapper(c)
    x = ensure_rank3(x)                       # [B,n,1]

    # 3) Compute N0
    no = ebnodb2no(ebno_db, num_bits_per_symbol=1, coderate=coderate)  # [B]
    no_sym = tf.reshape(no, [-1, 1, 1])                                # [B,1,1]

    # 4) Rayleigh + AWGN
    h = sample_rayleigh_block_fading(batch_size, n_no_rm, complex_channel=True, dtype=tf.float32)
    y = h * tf.cast(x, tf.complex64)
    y = add_awgn(y, no_sym, complex_noise=True)

    # 5) LLR computation
    y = tf.cast(y, tf.complex64)
    h = tf.cast(h, tf.complex64)
    llrs_in = llr_perfect_csi(y, h, no_sym)  # [B,n,1]
    h_context = tf.square(tf.abs(h))

    # 6) GNN forward + loss
    with tf.GradientTape() as tape:
        # --- FIX 1: Handle the list output from the GNN ---
        # gnn_decoder returns a LIST of 10 tensors, each with shape [B, N]
        c_hat_list = gnn_decoder((llrs_in, h_context))

        # Stack the list to create a single [B, T, N] tensor
        # T (num_iter) is 10
        c_hat_iter = tf.stack(c_hat_list, axis=1) 
        # ------------------------------------------------

        # 🔹 Repeat labels for loss calculation
        c_rep = tf.expand_dims(tf.cast(c, tf.float32), axis=1)  # [B,1,N]
        c_rep = tf.tile(c_rep, [1, num_iter, 1])                # [B,T,N]

        loss = bce(c_rep, c_hat_iter)

    # 7) Backprop
    grads = tape.gradient(loss, gnn_decoder.trainable_weights)
    optimizer.apply_gradients(zip(grads, gnn_decoder.trainable_weights))
    return loss

# ===============================
#  EVAL STEP (fixed)
# ===============================
@tf.function(jit_compile=True)
def eval_step(batch_size, ebno_db):
    """Evaluate BER for fixed Eb/N0 under Rayleigh block fading."""
    b = binary_source([batch_size, k_no_rm])
    c = encoder_no_rm(b)
    x = mapper(c)
    x = ensure_rank3(x)

    no = ebnodb2no(ebno_db, num_bits_per_symbol=1, coderate=coderate)
    no_sym = tf.reshape(no, [1, 1, 1])

    h = sample_rayleigh_block_fading(batch_size, n_no_rm, complex_channel=True, dtype=tf.float32)
    y = h * tf.cast(x, tf.complex64)
    y = add_awgn(y, no_sym, complex_noise=True)

    y = tf.cast(y, tf.complex64)
    h = tf.cast(h, tf.complex64)
    llrs_in = llr_perfect_csi(y, h, no_sym)

    h_context = tf.square(tf.abs(h))
    
    # --- FIX 2: Handle the list output from the GNN ---
    # gnn_decoder returns a LIST of 10 tensors, each with shape [B, N]
    c_hat_list = gnn_decoder((llrs_in, h_context))

    # For evaluation, we only need the output from the *final* iteration
    c_hat_llrs = c_hat_list[-1]  # Shape is [B, N]
    # ------------------------------------------------

    c_hat = tf.cast(c_hat_llrs < 0.0, tf.int32)
    ber = compute_ber(tf.cast(c, tf.int32), c_hat)
    return ber

# ===============================
#  TRAINING LOOP
# ===============================
print("Starting training...")
print(f"Code: {params['code']} (n={n_no_rm}, k={k_no_rm})")
print(f"GNN Iterations: {num_iter}")
print("Training on Rayleigh block fading (analytic)\n")

train_batch_size = params["batch_size"][0]
eval_batch_size  = params["batch_size_eval"]
ebno_db_train    = params["ebno_db_train"]    # e.g., [2.0, 8.0]
ebno_db_eval     = params["ebno_db_eval"]     # e.g., 2.0
total_train_iter = params["train_iter"][0]
eval_steps       = params["eval_train_steps"]
save_steps       = params["save_weights_iter"]

start_time = time.time()

for i in range(1, total_train_iter + 1):
    loss = train_step(train_batch_size, ebno_db_train)
    loss_val = float(loss.numpy())

    if i % 100 == 0:
        elapsed = time.time() - start_time
        print(f"Iter: {i:6d} | Loss: {loss_val:.4f} | Time: {elapsed:.1f}s")
        start_time = time.time()

    if i % eval_steps == 0:
        print(f"\n--- Evaluating model at Eb/No = {ebno_db_eval} dB ---")
        num_eval_batches = 100
        ber_vals = []
        for _ in range(num_eval_batches):
            ber = eval_step(eval_batch_size, ebno_db_eval)
            ber_vals.append(float(ber.numpy()))
        ber_avg = sum(ber_vals) / len(ber_vals)
        print(f"Iter: {i:6d} | EVAL BER: {ber_avg:.6e}")
        print("--------------------------------------------------")

    if i % save_steps == 0:
        save_path = ckpt_manager.save(checkpoint_number=i)
        print(f"Iter: {i:6d} | Saved checkpoint to {save_path}")

print("Training finished successfully ✅")